In [752]:
import pandas as pd
from sodapy import Socrata
from sqlalchemy import create_engine, inspect
import sqlite3
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import shapely.geometry 
from shapely.geometry import Point, Polygon
import os
import numpy as np

# Import SQL File

In [753]:
engine = create_engine("sqlite:///inspections.db", echo=False)

# connect to the sql database
conn = sqlite3.connect('inspections.db')

# create the ability to perform logic on the database
c = conn.cursor()

# create the table in the database
c.execute('CREATE TABLE IF NOT EXISTS food_inspections '\
          '(inspection_id TEXT, dba_name TEXT, aka_name TEXT, license_ TEXT, facility_type TEXT, risk TEXT, address TEXT, '\
          'city TEXT, state TEXT, zip TEXT, inspection_date TEXT, inspection_type TEXT, results TEXT, violations TEXT, '\
          'R1_1 TEXT, R1_2 TEXT, R1_3 TEXT, R2_1 TEXT, R2_2 TEXT, R2_3 TEXT, RS_1 TEXT, RS_2 TEXT, RS_3 TEXT, '\
          'latitude TEXT, longituide TEXT, locations TEXT, '\
          'PRIMARY KEY (inspection_id))')
# commit the stock table
conn.commit()

# print table name
print(inspect(engine).get_table_names())

['food_inspections']


In [754]:
df_new= pd.read_sql('SELECT* FROM food_inspections', engine)

# EDA

In [755]:
df_new["latitude"]=df_new["latitude"].astype(float)
df_new["longitude"]=df_new["longitude"].astype(float)


In [756]:
df_new.drop(df_new[df_new["results"]=="Out of Business"].index, inplace=True)
df_new.drop(df_new[df_new["results"]=="No Entry"].index, inplace=True) 
df_new.drop(df_new[df_new["results"]=="Not Ready"].index, inplace=True)
df_new.drop(df_new[df_new["results"]=="Out of Business"].index, inplace=True)
df_new.drop(df_new[df_new["results"]=="Business Not Located"].index, inplace=True)

In [757]:
df_new["results_int"]=0
df_new["results_int"].mask(df_new["results"]== "Pass", 1, inplace= True)
df_new["results_int"].mask(df_new["results"]== "Fail", 0, inplace= True)
df_new["results_int"].mask(df_new["results"]== "Pass w/ Conditions", 2, inplace= True)

In [758]:
df_new.drop(df_new[df_new["city"]!="CHICAGO"].index, inplace=True)

In [759]:
df_new= df_new[df_new["aka_name"].notna()]
df_new= df_new[df_new["license_"] != 0]
df_new= df_new[df_new["license_"].notna()]
df_new= df_new[df_new["facility_type"].notna()]
df_new= df_new[df_new["latitude"].notna()]
df_new= df_new[df_new["longitude"].notna()]

In [760]:
df_new["fail"]= np.where(df_new["results"]=="Fail", 1, 0)

In [761]:
df_new["results_int"]=df_new["results_int"].astype(int)
df_new["inspection_id"]=df_new["inspection_id"].astype(int)
df_new["zip"]=df_new["zip"].astype(int)

In [762]:
df_new[["year_in", "month_in", "time_in"]]= df_new.inspection_date.str.split('-', expand=True)
df_new= df_new.drop('time_in', 1)

/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [763]:
df_new["facility_group"]= df_new["facility_type"]

In [764]:
df_new=df_new.replace({"facility_group":{"ADULT DAYCARE":"Adult Care",
                                             "Assisted Living Senior Care":"Adult Care",
                                             "ASSISTED LIVING":"Adult Care",
                                             "Assisted Living":"Adult Care",
                                             "NURSING HOME":"Adult Care",
                                             "REHAB CENTER":"Adult Care",
                                             "SENIOR DAY CARE":"Adult Care",
                                             "SUPPORTIVE LIVING FACILITY":"Adult Care",
                                             "SUPPORTIVE LIVING":"Adult Care",
                                             "Long Term Care":"Adult Care",
                                             "Banquet Dining":"Banquet/Catering",
                                             "BANQUET FACILITY":"Banquet/Catering",
                                             "Banquet Hall":"Banquet/Catering",
                                             "BANQUET HALL":"Banquet/Catering",
                                             "BANQUET HALL/CATERING":"Banquet/Catering",
                                             "BANQUET ROOM":"Banquet/Catering",
                                             "BANQUET":"Banquet/Catering",
                                             "Banquet":"Banquet/Catering",
                                             "Banquet/kitchen":"Banquet/Catering",
                                             "banquets":"Banquet/Catering",
                                             "CATERING/BANQUET":"Banquet/Catering",
                                             "PRIVATE EVENT SPACE":"Banquet/Catering",
                                             "RESTAURANT.BANQUET HALLS":"Banquet/Catering",
                                             "Catering":"Banquet/Catering",
                                             "CATERING/CAFE":"Banquet/Catering",
                                             "EVENT CENTER":"Banquet/Catering",
                                             "EVENT SPACE":"Banquet/Catering",
                                             "EVENT VENU":"Banquet/Catering",
                                             "Lounge":"Banquet/Catering",
                                             "LOUNGE/BANQUET HALL":"Banquet/Catering",
                                             "PRIVATE CLUB":"Banquet/Catering",
                                             "Special Event":"Banquet/Catering",
                                             "NIGHT CLUB":"Banquet/Catering", 
                                             "Bakery":"Candy/Bakery/Ice Cream",
                                             "CANDY SHOP":"Candy/Bakery/Ice Cream",
                                             "CANDY STORE":"Candy/Bakery/Ice Cream",
                                             "CANDY/GELATO":"Candy/Bakery/Ice Cream",
                                             "DELI/BAKERY":"Candy/Bakery/Ice Cream",
                                             "donut shop":"Candy/Bakery/Ice Cream",
                                             "FROZEN DESSERT PUSHCARTS":"Candy/Bakery/Ice Cream",
                                             "ICE CREAM SHOP":"Candy/Bakery/Ice Cream",
                                             "ICE CREAM":"Candy/Bakery/Ice Cream",
                                             "PALETERIA":"Candy/Bakery/Ice Cream", 
                                             "1023 CHILDERN'S SERVICE FACILITY":"Childcare/School",
                                             "1023 CHILDERN'S SERVICES FACILITY":"Childcare/School",
                                             "1023 CHILDREN'S SERVICES FACILITY":"Childcare/School",
                                             "1023-CHILDREN'S SERVICES FACILITY":"Childcare/School",
                                             "CHILDERN'S SERVICE FACILITY":"Childcare/School",
                                             "Children's Services Facility":"Childcare/School",
                                             "15 monts to 5 years old":"Childcare/School",
                                             "AFTER SCHOOL PROGRAM":"Childcare/School",
                                             "CHARTER SCHOOL CAFETERIA":"Childcare/School",
                                             "CHARTER SCHOOL":"Childcare/School",
                                             "CHARTER":"Childcare/School",
                                             "DAY CARE 2-14":"Childcare/School",
                                             "Day Care Facility":"Childcare/School",
                                             "Daycare (2 - 6 Years)":"Childcare/School",
                                             "Daycare (Under 2 Years)":"Childcare/School",
                                             "DAYCARE 2 YRS TO 12 YRS":"Childcare/School",
                                             "Daycare Above and Under 2 Years":"Childcare/School",
                                             "Daycare Combo 1586":"Childcare/School",
                                             "Daycare Night":"Childcare/School",
                                             "DAYCARE":"Childcare/School",
                                             "PRIVATE SCHOOL":"Childcare/School",
                                             "private school":"Childcare/School",
                                             "Private School":"Childcare/School",
                                             "PUBLIC SHCOOL":"Childcare/School",
                                             "school cafeteria":"Childcare/School",
                                             "School":"Childcare/School",
                                             "SCHOOL":"Childcare/School",
                                             "TEACHING SCHOOL":"Childcare/School",
                                             "youth housing":"Childcare/School",
                                             "PREP INSIDE SCHOOL":"Childcare/School",
                                             "UNIVERSITY CAFETERIA":"Childcare/School",
                                             "COFFEE  SHOP":"Coffee/Beverage Shop",
                                             "COFFEE KIOSK":"Coffee/Beverage Shop",
                                             "coffee shop":"Coffee/Beverage Shop",
                                             "Coffee shop":"Coffee/Beverage Shop",
                                             "COFFEE SHOP":"Coffee/Beverage Shop",
                                             "CAFE/STORE":"Coffee/Beverage Shop",
                                             "JUICE AND SALAD BAR":"Coffee/Beverage Shop",
                                             "JUICE BAR":"Coffee/Beverage Shop",
                                             "MILK TEA":"Coffee/Beverage Shop",
                                             "SHAKES/TEAS":"Coffee/Beverage Shop",
                                             "smoothie bar":"Coffee/Beverage Shop",
                                             "Internet Cafe":"Coffee/Beverage Shop",
                                             "CONVENIENCE STORE": "Gas Station/Convenience",
                                             "convenience store": "Gas Station/Convenience",
                                             "convenience": "Gas Station/Convenience",
                                             "convenience/drug store": "Gas Station/Convenience",
                                             "CONVENIENCE/GAS STATION": "Gas Station/Convenience",
                                             "CONVENIENT STORE": "Gas Station/Convenience",
                                             "GAS STATION FOOD STORE": "Gas Station/Convenience",
                                             "GAS STATION STORE": "Gas Station/Convenience",
                                             "GAS STATION": "Gas Station/Convenience",
                                             "gas station": "Gas Station/Convenience",
                                             "GAS STATION/CONVENIENCE STORE": "Gas Station/Convenience",
                                             "GAS STATION/FOOD": "Gas Station/Convenience",
                                             "GAS STATION/GROCERY": "Gas Station/Convenience",
                                             "GAS STATION/MINI MART": "Gas Station/Convenience",
                                             "GAS STATION/RESTAURANT": "Gas Station/Convenience",
                                             "GAS STATION/STORE": "Gas Station/Convenience",
                                             "CHURCH (SPECIAL EVENTS)":"Hospital/Church/Community",
                                             "CHURCH KITCHEN":"Hospital/Church/Community",
                                             "CHURCH":"Hospital/Church/Community",
                                             "Church":"Hospital/Church/Community",
                                             "CHURCH/SPECIAL EVENTS":"Hospital/Church/Community",
                                             "Hospital":"Hospital/Church/Community",
                                             "NOT-FOR-PROFIT CLUB":"Hospital/Church/Community",
                                             "CHARITY AID KITCHEN":"Hospital/Church/Community",
                                             "Shelter":"Hospital/Church/Community",
                                             "HOSTEL": "Hospitality",
                                             "HOTEL": "Hospitality",
                                             "1475 LIQUOR": "Liqour Sale/Bar",
                                             "BAR": "Liqour Sale/Bar",
                                             "bar": "Liqour Sale/Bar",
                                             "BREWERY": "Liqour Sale/Bar",
                                             "BREWPUB": "Liqour Sale/Bar",
                                             "CATERED LIQUOR": "Liqour Sale/Bar",
                                             "DISTILLERY WITH TASTING ROOM": "Liqour Sale/Bar",
                                             "LIQOUR BREWERY TASTING": "Liqour Sale/Bar",
                                             "liquor store": "Liqour Sale/Bar",
                                             "Liquor": "Liqour Sale/Bar",
                                             "LIQUOR/COFFEE KIOSK": "Liqour Sale/Bar",
                                             "MOBIL FOOD PREPARED": "Mobile Food",
                                             "MOBILE DESSERTS VENDOR": "Mobile Food",
                                             "Mobile Food Dispenser": "Mobile Food",
                                             "Mobile Food Preparer": "Mobile Food",
                                             "MOBILE FOOD TRUCK": "Mobile Food",
                                             "MOBILE FOOD": "Mobile Food",
                                             "Mobile Frozen Desserts Vendor": "Mobile Food",
                                             "Mobile Prepared Food Vendor": "Mobile Food",
                                             "MOBILE PUSH CART": "Mobile Food",
                                             "MOBILPREPARED FOOD VENDOR": "Mobile Food",
                                             "FLEA MARKET": "Mobile Food",
                                             "FOOD BOOTH": "Mobile Food",
                                             "POPCORN CORN": "Mobile Food",
                                             "Navy Pier Kiosk": "Mobile food",
                                             "Animal Shelter Cafe Permit": "Other",
                                             "Airport Lounge": "Other",
                                             "ARCHDIOCESE": "Other",
                                             "BEVERAGE/SILVERWARE WAREHOUSE": "Other",
                                             "CAFETERIA": "Other",
                                             "Cafeteria": "Other",
                                             "CHILDRENS SERVICES FACILITY": "Other",
                                             "CLOTHING STORE": "Other",
                                             "COLD/FROZEN FOOD STORAGE": "Other",
                                             "COMMISSARY": "Other",
                                             "DISTRIBUTION CENTER": "Other",
                                             "Food Pantry": "Other",
                                             "FRENCH MARKET SPACE": "Other",
                                             "FURNITURE STORE": "Other",
                                             "Gift Shop": "Other",
                                             "GOLF COURSE CONNCESSION STAND": "Other",
                                             "GREENHOUSE": "Other",
                                             "hair salon": "Other",
                                             "Hair Salon": "Other",
                                             "HAIR SALON": "Other",
                                             "HELICOPTER TERMINAL": "Other",
                                             "HOT DOG STATION": "Other",
                                             "KIOSK": "Other",
                                             "KITCHEN DEMO": "Other",
                                             "KITCHEN": "Other",
                                             "Laundromat": "Other",
                                             "MOVIE THEATER": "Other",
                                             "MOVIE THEATRE": "Other",
                                             "Other": "Other",
                                             "OUTREACH CULINARY KITCHEN": "Other",
                                             "PACKAGED HEALTH FOODS": "Other",
                                             "PRE PACKAGED": "Other",
                                             "REGULATED BUSINESS": "Other",
                                             "REPACKAGING PLANT": "Other",
                                             "Shared Kitchen User (Long Term)": "Other",
                                             "Shared Kitchen User (Short Term)": "Other",
                                             "Shared Kitchen": "Other",
                                             "Shuffleboard Club with Bar": "Other",
                                             "STADIUM": "Other",
                                             "TEST KITCHEN/ STORAGE": "Other",
                                             "THEATER": "Other",
                                             "THEATRE": "Other",
                                             "TOBACCO STORE": "Other",
                                             "UNUSED STORAGE": "Other",
                                             "VENDING COMMISSARY": "Other",
                                             "WAREHOUSE": "Other",
                                             "ART CENTER": "Professional School",
                                             "A-Not-For-Profit Chef Training Program": "Professional School",
                                             "cooking school": "Professional School",
                                             "COOKING SCHOOL": "Professional School",
                                             "CULINARY SCHOOL": "Professional School",
                                             "PASTRY SCHOOL": "Professional School",
                                             "PASTRY school": "Professional School",
                                             "Restaurant": "Resturant",
                                             "RESTAURANT/BAR": "Resturant",
                                             "RESTAURANT/BAR/THEATER": "Resturant",
                                             "RIVERWALK": "Resturant",
                                             "ROOF TOP": "Resturant",
                                             "ROOF TOPS": "Resturant",
                                             "ROOFTOP": "Resturant",
                                             "ROOFTOPS": "Resturant",
                                             "SUSHI COUNTER": "Resturant",
                                             "TAVERN": "Resturant",
                                             "tavern": "Resturant",
                                             "TAVERN/STORE": "Resturant",
                                             "Wrigley Roof Top": "Resturant",
                                             "WRIGLEY ROOFTOP": "Resturant",
                                             "COMMISARY RESTAURANT": "Resturant",
                                             "DINING HALL": "Resturant",
                                             "Golden Diner": "Resturant",
                                             "Pop-Up Establishment Host-Tier II": "Resturant",
                                             "Pop-Up Establishment Host-Tier III": "Resturant",
                                             "Pop-Up Food Establishment User-Tier II": "Resturant",
                                             "Pop-Up Food Establishment User-Tier III": "Resturant",
                                             "RETAIL STORE OFFERS COOKING CLASSES": "Retail Store",
                                             "RETAIL": "Retail Store",
                                             "LIVE POULTRY SLAUGHTER FACILITY": "Slaughter/Live ",
                                             "LIVE POULTRY": "Slaughter/Live ",
                                             "Live Poultry": "Slaughter/Live ",
                                             "POULTRY SLAUGHTER": "Slaughter/Live ",
                                             "Poultry Slaughter": "Slaughter/Live",
                                             "SLAUGHTER HOUSE/ GROCERY": "Slaughter/Live",
                                             "BUTCHER SHOP": "Slaughter/Live",
                                             "butcher shop": "Slaughter/Live",
                                             "CUSTOM POULTRY SLAUGHTER": "Slaughter/Live",
                                             "SUMMER FEEDING PREP AREA": "Slaughter/Live",
                                             'Daycare':"Childcare/School",
                                             "DOLLAR & GROCERY STORE'": "Grocery store",
                                             "DOLLAR STORE WITH GROCERY'": "Grocery store",
                                             "DOLLAR STORE'": "Grocery store",
                                             "DOLLAR TREE'": "Grocery store",
                                             "DRUG STORE'": "Grocery store",
                                             "DRUG STORE/GROCERY'": "Grocery store",
                                             "GROCERY & LIQUOR STORE'": "Grocery store",
                                             "Grocery & Restaurant'": "Grocery store",
                                             "GROCERY AND BUTCHER'": "Grocery store",
                                             "GROCERY STORE /PHARMACY'": "Grocery store",
                                             "Grocery Store'": "Grocery store",
                                             "GROCERY STORE'": "Grocery store",
                                             "GROCERY STORE/BAKERY'": "Grocery store",
                                             "GROCERY STORE/COOKING SCHOOL'": "Grocery store",
                                             "GROCERY STORE/GAS STATION'": "Grocery store",
                                             "GROCERY(GAS STATION)'": "Grocery store",
                                             "Grocery(Sushi prep)'": "Grocery store",
                                             "GROCERY/BAKERY'": "Grocery store",
                                             "GROCERY/DELI'": "Grocery store",
                                             "grocery/dollar store'": "Grocery store",
                                             "GROCERY/DRUG STORE'": "Grocery store",
                                             "GROCERY/GAS STATION'": "Grocery store",
                                             "GROCERY/RESTAURANT'": "Grocery store",
                                             "GROCERY/TAQUERIA'": "Grocery store",
                                             "GROCERY& RESTAURANT'": "Grocery store",
                                             "JUICE BAR/GROCERY'": "Grocery store",
                                             "LIQUOR/GROCERY STORE/BAR'": "Grocery store",
                                             "REST/GROCERY']": "Grocery store",
                                             "RESTAURANT/GROCERY STORE'": "Grocery store",
                                             "DELI/GROCERY STORE'": "Grocery Store",
                                             "STORE'": "Grocery Store",
                                             "WHOLESALE & RETAIL'": "Grocery Store",
                                             "Wholesale'": "Grocery Store",
                                             'Grocery(Sushi prep)': "Grocery Store",
                                             "Grocery & Restaurant": "Grocery Store",
                                             "GROCERY & LIQUOR STORE'": "Grocery Store",
                                             "GROCERY/GAS STATION": "Grocery Store",
                                             "GROCERY/TAQUERIA": "Grocery Store",
                                             "GROCERY/RESTAURANT": "Grocery Store",
                                             "DOLLAR STORE WITH GROCERY": "Grocery Store",
                                             "GROCERY/GAS STATION": "Grocery Store",
                                             'GROCERY STORE/COOKING SCHOOL':"Grocery Store",
                                             'Grocery & Restaurant':"Grocery Store",
                                             'GROCERY/BAKERY':"Grocery Store",
                                             'GROCERY STORE':"Grocery Store",
                                             'DOLLAR STORE WITH GROCERY':"Grocery Store",
                                             'LIQUOR/GROCERY STORE/BAR':"Grocery Store",
                                             'DOLLAR STORE':"Grocery Store",
                                             'DOLLAR TREE':"Grocery Store",
                                             'RESTAURANT/GROCERY STORE':"Grocery Store",
                                             'STORE':"Grocery Store",
                                             'GROCERY STORE /PHARMACY':"Grocery Store",
                                             'GROCERY& RESTAURANT':"Grocery Store",
                                             'GROCERY/DRUG STORE':"Grocery Store",
                                             'GROCERY AND BUTCHER':"Grocery Store",
                                             'GROCERY(GAS STATION)':"Grocery Store",
                                             'DELI/GROCERY STORE':"Grocery Store",
                                             'Resturant ':"Resturant",
                                             'Grocery(Sushi prep)':"Grocery Store",
                                             'Wholesale':"Other",
                                             'GROCERY STORE/GAS STATION':"Grocery Store",
                                             'GROCERY & LIQUOR STORE':"Grocery Store",
                                             'GROCERY/GAS STATION':"Grocery Store",
                                             'GROCERY/TAQUERIA':"Grocery Store",
                                             'REST/GROCERY':"Grocery Store",
                                             'Rest/GYM':"Herbal/Health/Fitness Store",
                                             'HERBAL LIFE SHOP':"Herbal/Health/Fitness Store",
                                             'HEALTH FOOD STORE':"Herbal/Health/Fitness Store",
                                             'JUICE BAR/GROCERY':"Herbal/Health/Fitness Store",
                                             "GYM":"Herbal/Health/Fitness Store",
                                             'FITNESS STUDIO':"Herbal/Health/Fitness Store",
                                             'Herbalife Nutrition':"Herbal/Health/Fitness Store",
                                             'Herabalife':"Herbal/Health/Fitness Store",
                                             'HERBALIFE/ZUMBA':"Herbal/Health/Fitness Store",
                                             'FITNESS CENTER':"Herbal/Health/Fitness Store",
                                             'Herbalife Nutrition':"Herbal/Health/Fitness Store",
                                             'HERBAL LIFE':"Herbal/Health/Fitness Store",
                                             'HERBALIFE':"Herbal/Health/Fitness Store",
                                             'WEIGHT LOSS PROGRAM':"Herbal/Health/Fitness Store",
                                             'NUTRITION SHAKES':"Herbal/Health/Fitness Store",
                                             'HERBALIFE STORE':"Herbal/Health/Fitness Store",
                                             'WEIGHT LOSS PROGRAM':"Herbal/Health/Fitness Store",
                                             'GYM STORE':"Herbal/Health/Fitness Store",
                                             'HERBAL DRINKS':"Herbal/Health/Fitness Store",
                                             'HEALTH CARE STORE':"Herbal/Health/Fitness Store",
                                             'SPA':"Herbal/Health/Fitness Store",
                                             'Restaurant(protein shake bar)':"Herbal/Health/Fitness Store",
                                             'GROCERY/DELI':"Grocery Store",
                                             'Slaughter/Live ':"Slaughter/Live",
                                             'GROCERY/DELI':"Grocery Store",
                                             'GROCERY STORE/BAKERY':"Grocery Store",
                                             'DRUG STORE/GROCERY':"Gas Station/Convenience",
                                             'DRUG STORE':"Gas Station/Convenience",
                                             'employee kitchen':"Other",
                                             'WHOLESALE & RETAIL':"Other",
                                             'DOLLAR & GROCERY STORE':"Grocery Store",
                                             'grocery/dollar store':"Grocery Store",
                                             }})


# Total Inspections by Zip Code

In [765]:
zip_df= df_new.groupby(["zip"],as_index=False).results.count()
zip_df=zip_df.sort_values(by= ["results"], ascending = False)
zip_df.rename(columns= {"results": "total_count"}, inplace= True)

zip_df

,zip,total_count
13,60614,2721
44,60647,2607
52,60657,2428
24,60625,2295
17,60618,2248
37,60640,2187
10,60611,2153
6,60607,2119
7,60608,2065
36,60639,1981


# Fail Data


In [766]:
df_fail= df_new[df_new['results']=="Fail"]
df_fail

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,inspection_type,results,violations,latitude,longitude,results_int,fail,year_in,month_in,facility_group
2,2554086,GOLDEN STAR TOBACCO & GROCERY INC,GOLDEN STAR TOBACCO & GROCERY INC,2835873,Grocery Store,Risk 3 (Low),8637 S COTTAGE GROVE AVE,CHICAGO,IL,60619,...,License,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.737361,-87.604696,0,1,2022,04,Grocery Store
3,2554099,PRESENCE CHICAGO HOSPITALS NETWORK,ST. JOSEPH HOSPITAL,1144381,Restaurant,Risk 1 (High),2900 N LAKE SHORE DR,CHICAGO,IL,60657,...,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.934403,-87.636806,0,1,2022,04,Resturant
5,2554098,"CITIZENS CHICAGO, LLC",CITIZENS CHICAGO,2840575,Restaurant,Risk 2 (Medium),333 S WABASH AVE,CHICAGO,IL,60604,...,License,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.877450,-87.625922,0,1,2022,04,Resturant
9,2554030,CHABAD DAY CARE,CHABAD DAY CARE,2818380,Daycare Above and Under 2 Years,Risk 1 (High),405 W SUPERIOR ST,CHICAGO,IL,60654,...,License,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.895422,-87.638793,0,1,2022,04,Childcare/School
10,2553998,FRANK W REILLY ELEMENTARY SCHOOL,FRANK W. REILLY ELEMENTARY SCHOOL,25101,School,Risk 1 (High),3650 W School ST,CHICAGO,IL,60618,...,Canvass,Fail,16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...,41.941122,-87.719531,0,1,2022,04,Childcare/School
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73219,1953819,FOUR FARTHINGS TAVERN & GRILL,FOUR FARTHINGS TAVERN & GRILL,691,Restaurant,Risk 1 (High),2058-2060 N CLEVELAND AVE,CHICAGO,IL,60614,...,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.920016,-87.641449,0,1,2016,09,Resturant
73223,1953791,JERSEY MIKE'S SUBS,JERSEY MIKE'S SUBS,2398227,Restaurant,Risk 1 (High),200 W MONROE ST,CHICAGO,IL,60606,...,Canvass,Fail,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.880769,-87.633875,0,1,2016,09,Resturant
73225,1953823,FRESH SMOOTHIES CHI,FRESH SMOOTHIES CHI,2469332,Mobile Food Preparer,Risk 2 (Medium),324 N LEAVITT ST,CHICAGO,IL,60612,...,License,Fail,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.887434,-87.681849,0,1,2016,09,Mobile Food
73233,1953822,JEWEL FOOD STORE #3241,JEWEL FOOD STORE #3241,1398,Grocery Store,Risk 1 (High),1341 N PAULINA ST,CHICAGO,IL,60622,...,Complaint,Fail,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.906244,-87.669862,0,1,2016,09,Grocery Store


# Fail Count by Zipcode

In [767]:
df_fail_zip= df_fail.groupby("zip", as_index=False)["results"].agg("count")

df_fail_zip=df_fail_zip.sort_values(by= ["results"], ascending = False)
df_fail_zip.rename(columns= {"results": "fail_count"}, inplace= True)
df_fail_zip

,zip,fail_count
44,60647,804
24,60625,708
13,60614,672
36,60639,651
37,60640,642
52,60657,602
22,60623,564
7,60608,553
6,60607,540
19,60620,530


# Pass Data

In [768]:
df_pass= df_new[df_new['results']=="Pass"]
df_pass

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,inspection_type,results,violations,latitude,longitude,results_int,fail,year_in,month_in,facility_group
0,2554087,Tropical Smoothie Cafe,Tropical Smoothie Cafe,2840460,Restaurant,Risk 2 (Medium),4874 N LINCOLN AVE,CHICAGO,IL,60625,...,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.970550,-87.689937,1,0,2022,04,Resturant
1,2554079,MOCHINUT,MOCHINUT,2827751,Restaurant,Risk 2 (Medium),1139 W TAYLOR ST,CHICAGO,IL,60607,...,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.869323,-87.655748,1,0,2022,04,Resturant
4,2554096,EPIC KITCHENS,EPIC KITCHENS,2840582,Restaurant,Risk 1 (High),1250 S MICHIGAN AVE,CHICAGO,IL,60605,...,License,Pass,"6. PROPER EATING, TASTING, DRINKING, OR TOBACC...",41.866074,-87.624268,1,0,2022,04,Resturant
8,2554012,CHEMISTRY,CHEMISTRY,2803011,Restaurant,Risk 1 (High),5115-5121 S HARPER AVE,CHICAGO,IL,60615,...,License Re-Inspection,Pass,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...,41.801951,-87.588998,1,0,2022,04,Resturant
12,2554016,Chicago Prep Master Tenant LLC,Chicago Prep Master Tenant,2723671,Shared Kitchen,Risk 1 (High),5801 W Dickens AVE,CHICAGO,IL,60639,...,Canvass Re-Inspection,Pass,51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...,41.917869,-87.770697,1,0,2022,04,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73231,1953844,CAFFE' OLIVA,CAFFE' OLIVA,2152750,Restaurant,Risk 1 (High),550 E GRAND AVE,CHICAGO,IL,60611,...,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.892064,-87.612534,1,0,2016,09,Resturant
73232,1953865,STARBUCKS COFFEE #2229,STARBUCKS COFFEE #2229,23085,Restaurant,Risk 2 (Medium),209 W JACKSON BLVD,CHICAGO,IL,60606,...,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.878006,-87.634022,1,0,2016,09,Resturant
73234,1953816,NEW ENGLAND SEAFOOD COMPANY FL,NEW ENGLAND SEAFOOD COMPANY,2108692,Restaurant,Risk 1 (High),3341 N LINCOLN AVE,CHICAGO,IL,60657,...,Canvass Re-Inspection,Pass,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.942573,-87.670513,1,0,2016,09,Resturant
73236,1953693,CVS/PHARMACY #8753,CVS/PHARMACY #8753,1695423,Grocery Store,Risk 3 (Low),3637 N SOUTHPORT AVE,CHICAGO,IL,60613,...,Canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.948179,-87.663897,1,0,2016,09,Grocery Store


# Pass Count by Zipcode

In [769]:
df_pass_zip= df_pass.groupby("zip", as_index=False)["results"].agg("count")

df_pass_zip=df_pass_zip.sort_values(by= ["results"], ascending = False)
df_pass_zip.rename(columns= {"results": "pass_count"}, inplace= True)
df_pass_zip

,zip,pass_count
13,60614,1131
17,60618,1042
52,60657,1019
10,60611,999
44,60647,967
6,60607,943
21,60622,870
7,60608,856
24,60625,821
15,60616,820


# Pass with Conditions Data

In [770]:
df_pass_w_cond= df_new[df_new['results']=="Pass w/ Conditions"]
df_pass_w_cond_zip= df_pass_w_cond.groupby("zip", as_index=False)["results"].agg("count")
df_pass_w_cond_zip=df_pass_w_cond_zip.sort_values(by= ["results"], ascending = False)
df_pass_w_cond_zip.rename(columns= {"results": "pass__w_cond_count"}, inplace= True)
df_pass_w_cond_zip

,zip,pass__w_cond_count
13,60614,918
44,60647,836
10,60611,808
52,60657,807
37,60640,772
24,60625,766
17,60618,720
21,60622,689
7,60608,656
6,60607,636


# Total Count by Zipcode

In [771]:
df_zip_total_count= df_pass_w_cond_zip.merge(df_fail_zip, left_on='zip', right_on='zip')
df_zip_total_count=df_zip_total_count.merge(df_pass_zip, left_on='zip', right_on='zip')
df_zip_total_count=df_zip_total_count.merge(zip_df, left_on='zip', right_on='zip')
df_zip_total_count["percent_fail"]=df_zip_total_count["fail_count"]/df_zip_total_count["total_count"]*100
df_zip_total_count["percent_pass"]=df_zip_total_count["pass_count"]/df_zip_total_count["total_count"]*100
df_zip_total_count["percent_pass_w_con"]=df_zip_total_count["pass__w_cond_count"]/df_zip_total_count["total_count"]*100
df_zip_total_count["zip"]=df_zip_total_count["zip"].astype(int)
df_zip_total_count

,zip,pass__w_cond_count,fail_count,pass_count,total_count,percent_fail,percent_pass,percent_pass_w_con
0,60614,918,672,1131,2721,24.696803,41.565601,33.737596
1,60647,836,804,967,2607,30.840046,37.092443,32.067511
2,60611,808,346,999,2153,16.070599,46.400372,37.529029
3,60657,807,602,1019,2428,24.794069,41.968699,33.237232
4,60640,772,642,773,2187,29.355281,35.345222,35.299497
5,60625,766,708,821,2295,30.849673,35.773420,33.376906
6,60618,720,486,1042,2248,21.619217,46.352313,32.028470
7,60622,689,380,870,1939,19.597731,44.868489,35.533780
8,60608,656,553,856,2065,26.779661,41.452785,31.767554
9,60607,636,540,943,2119,25.483719,44.502124,30.014158


# Import IRS Adjusted Gross Income Data

In [772]:
AGI=pd.read_csv("AGI_IRS.csv")
AGI.head()

,zip,AGI
0,60002,"1,017,717"
1,60004,"3,028,778"
2,60005,"1,579,183"
3,60007,"1,401,910"
4,60008,"784,531"


In [773]:
df_zip_total_count=df_zip_total_count.merge(AGI, left_on='zip', right_on='zip')
df_zip_total_count

,zip,pass__w_cond_count,fail_count,pass_count,total_count,percent_fail,percent_pass,percent_pass_w_con,AGI
0,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797"
1,60647,836,804,967,2607,30.840046,37.092443,32.067511,"3,872,226"
2,60611,808,346,999,2153,16.070599,46.400372,37.529029,"6,375,314"
3,60657,807,602,1019,2428,24.794069,41.968699,33.237232,"5,493,694"
4,60640,772,642,773,2187,29.355281,35.345222,35.299497,"2,754,316"
5,60625,766,708,821,2295,30.849673,35.773420,33.376906,"2,567,308"
6,60618,720,486,1042,2248,21.619217,46.352313,32.028470,"4,012,068"
7,60622,689,380,870,1939,19.597731,44.868489,35.533780,"3,449,578"
8,60608,656,553,856,2065,26.779661,41.452785,31.767554,"1,511,740"
9,60607,636,540,943,2119,25.483719,44.502124,30.014158,"2,251,226"


In [815]:
df_new=df_zip_total_count.merge(df_new, left_on='zip', right_on='zip')
df_new

,zip,pass__w_cond_count_x,fail_count_x,pass_count_x,total_count_x,percent_fail_x,percent_pass_x,percent_pass_w_con_x,AGI_x,pass__w_cond_count_y,...,inspection_type,results,violations,latitude,longitude,results_int,fail,year_in,month_in,facility_group
0,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,License,Pass,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.932816,-87.649620,1,0,2022,04,Childcare/School
1,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,Canvass,Fail,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.929001,-87.642666,0,1,2022,04,Resturant
2,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,Suspected Food Poisoning,Pass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...,41.926985,-87.672103,1,0,2022,04,Resturant
3,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,Canvass,Fail,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.925097,-87.664078,0,1,2022,04,Resturant
4,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,Canvass,Pass,57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAIN...,41.913130,-87.652817,1,0,2022,04,Resturant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70876,60827,8,24,26,58,41.379310,44.827586,13.793103,"359,117",8,...,Canvass Re-Inspection,Pass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.653760,-87.614342,1,0,2016,10,Childcare/School
70877,60827,8,24,26,58,41.379310,44.827586,13.793103,"359,117",8,...,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.653637,-87.605071,0,1,2016,10,Childcare/School
70878,60827,8,24,26,58,41.379310,44.827586,13.793103,"359,117",8,...,Canvass,Fail,29. PREVIOUS MINOR VIOLATION(S) CORRECTED 7-42...,41.657432,-87.606496,0,1,2016,10,Childcare/School
70879,60827,8,24,26,58,41.379310,44.827586,13.793103,"359,117",8,...,Canvass,Fail,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,41.653760,-87.614342,0,1,2016,10,Childcare/School


In [ ]:
for 

In [811]:

df_new.to_csv("Inspections_df.csv")

In [814]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70881 entries, 0 to 70880
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zip                 70881 non-null  int64  
 1   pass__w_cond_count  70881 non-null  int64  
 2   fail_count          70881 non-null  int64  
 3   pass_count          70881 non-null  int64  
 4   total_count         70881 non-null  int64  
 5   percent_fail        70881 non-null  float64
 6   percent_pass        70881 non-null  float64
 7   percent_pass_w_con  70881 non-null  float64
 8   AGI                 70881 non-null  object 
 9   inspection_id       70881 non-null  int64  
 10  dba_name            70881 non-null  object 
 11  aka_name            70881 non-null  object 
 12  license_            70881 non-null  object 
 13  facility_type       70881 non-null  object 
 14  risk                70881 non-null  object 
 15  address             70881 non-null  object 
 16  city

# Unit Testing

In [823]:
expected = pd.Series([True, False], name="zip")
pd.testing.assert_series_equal(df_new["zip"], expected)



AssertionError: Series are different

Series length are different
[left]:  70881, Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            70871, 70872, 70873, 70874, 70875, 70876, 70877, 70878, 70879,
            70880],
           dtype='int64', length=70881)
[right]: 2, RangeIndex(start=0, stop=2, step=1)

In [824]:
import unittest
 
class TestStringMethods(unittest.TestCase):
    # test function
    def test_positive(self):
        testValue = True
        # error message in case if test case got failed
        message = "Test value is not true."
        # assertTrue() to check true of test value
        self.assertTrue( testValue, message)
 
if __name__ == '__main__':
    unittest.main()

E
ERROR: /Users/snizhanakurylyuk/Library/Jupyter/runtime/kernel-e867ff2b-317e-4e66-8bb2-de7fe5fea13c (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/snizhanakurylyuk/Library/Jupyter/runtime/kernel-e867ff2b-317e-4e66-8bb2-de7fe5fea13c'

----------------------------------------------------------------------
Ran 1 test in 0.008s

FAILED (errors=1)


SystemExit: True

/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [847]:
df_new[(df_new["results"] == "Fail") & (df_new["results_int"]==0)]

,zip,pass__w_cond_count_x,fail_count_x,pass_count_x,total_count_x,percent_fail_x,percent_pass_x,percent_pass_w_con_x,AGI_x,pass__w_cond_count_y,...,inspection_type,results,violations,latitude,longitude,results_int,fail,year_in,month_in,facility_group
1,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,Canvass,Fail,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.929001,-87.642666,0,1,2022,04,Resturant
3,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,Canvass,Fail,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.925097,-87.664078,0,1,2022,04,Resturant
5,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,Canvass,Fail,16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...,41.926213,-87.640707,0,1,2022,04,Resturant
6,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,Canvass,Fail,8. HANDS CLEAN & PROPERLY WASHED - Comments: O...,41.932864,-87.647246,0,1,2022,03,Resturant
12,60614,918,672,1131,2721,24.696803,41.565601,33.737596,"8,996,797",918,...,License,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.913755,-87.646010,0,1,2022,03,Grocery Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70875,60827,8,24,26,58,41.379310,44.827586,13.793103,"359,117",8,...,Canvass,Fail,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,41.656569,-87.597395,0,1,2016,10,Resturant
70877,60827,8,24,26,58,41.379310,44.827586,13.793103,"359,117",8,...,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.653637,-87.605071,0,1,2016,10,Childcare/School
70878,60827,8,24,26,58,41.379310,44.827586,13.793103,"359,117",8,...,Canvass,Fail,29. PREVIOUS MINOR VIOLATION(S) CORRECTED 7-42...,41.657432,-87.606496,0,1,2016,10,Childcare/School
70879,60827,8,24,26,58,41.379310,44.827586,13.793103,"359,117",8,...,Canvass,Fail,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,41.653760,-87.614342,0,1,2016,10,Childcare/School


In [848]:
df_new["results"].unique()

array(['Pass', 'Fail', 'Pass w/ Conditions'], dtype=object)

In [855]:

 

#def results_testing(i):
for i,j in enumerate(df_new["results"]):
    assert (df_new["results"][i] == "Fail" and  df_new["results_int"][i]==0) | (df_new["results"][i] == "Pass" and  df_new["results_int"][i]==1) |(df_new["results"][i] == "Pass w/ Conditions" and  df_new["results_int"][i]==2), "This did not pass" 

print("Test Pass")
        
            


AssertionError: This did not pass